# Initial Modelling
We are going to make some basic models to predict both Min and Max demands for a day using findings from the EDA phase of the projects.
Models will start simple using a few datasets to predict and then get more complex.
Models will be using RMSE and MAE to evaluate and compare.

In [1]:
# Import general packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Load Data and Format for Modelling
To begin with we will load in the:
- Total Demand Dataset
- NSW Temperature Dataset
- NSW Residential Solar Data
- NSW Population Data

Seeing as the Residential Solar Data only goes up to 2020 and Demand Data is only from 2010, when doing the train test split data from 2010-2018 will be used for training, 2019 and 2020 will be used for test. Other data will be discarded for now.

Data that have recording interval periods greater than 1 day will be linearly interpolated for now for simplicity.

In [2]:
# Import demand dataset
demand_df = pd.read_csv('../data/raw/totaldemand_nsw.csv', names=['datetime', 'region', 'demand'], header=0)
demand_df['datetime'] = pd.to_datetime(demand_df['datetime'])
demand_df = demand_df.resample('D', on='datetime')['demand'].agg(['min', 'max'])
demand_df.rename(columns={'min':'demand_min', 'max':'demand_max'}, inplace=True)
demand_df.head()

,demand_min,demand_max
datetime,,
2010-01-01,6157.36,8922.42
2010-01-02,6112.73,9326.64
2010-01-03,6014.91,8277.85
2010-01-04,6023.79,9522.30
2010-01-05,6287.12,10728.72


In [3]:
# Import temperature data
temp_df = pd.read_csv('../data/raw/temperature_nsw.csv', names=['datetime', 'location', 'temp'], header=0)
temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
temp_df.drop(temp_df[temp_df['temp'] <= -9999].index, inplace = True)
temp_df = temp_df.resample('D', on='datetime')['temp'].agg(['min', 'max', 'mean'])
temp_df.rename(columns={'min':'temp_min', 'max':'temp_max', 'mean':'temp_mean'}, inplace=True)
temp_df.head()

,temp_min,temp_max,temp_mean
datetime,,,
2010-01-01,22.1,28.8,25.094000
2010-01-02,21.6,29.4,24.765385
2010-01-03,17.9,21.5,19.429825
2010-01-04,17.9,23.9,20.625926
2010-01-05,15.4,27.7,22.660417


In [21]:
# Import solar data
solar_df = pd.read_csv('../data/raw/nsw_residential_solar.csv', names=['datetime', 'units', 'cum_units', 'output', 'cum_output'], header=0)
solar_df['datetime'] = pd.to_datetime(solar_df['datetime'])
# Interpolate to get daily data
solar_df = solar_df.set_index('datetime').resample('D', convention='end').interpolate(method='linear')
solar_df.head()

,units,cum_units,output,cum_output
datetime,,,,
2008-01-01,127.000000,1882.000000,287.946000,2.710745e+03
2008-01-02,128.451613,1887.548387,286.983677,2.719071e+03
2008-01-03,129.903226,1893.096774,286.021355,2.727398e+03
2008-01-04,131.354839,1898.645161,285.059032,2.735724e+03
2008-01-05,132.806452,1904.193548,284.096710,2.744050e+03
...,...,...,...,...
2020-11-27,11792.333333,677988.866667,110417.842533,3.493903e+06
2020-11-28,11777.000000,678379.900000,110718.932400,3.497624e+06
2020-11-29,11761.666667,678770.933333,111020.022267,3.501344e+06


In [5]:
# Import population data
pop_df = pd.read_csv('../data/raw/NSW_population.csv', usecols=['TIME_PERIOD: Time Period', 'OBS_VALUE'], header=0)
pop_df.rename(columns={'TIME_PERIOD: Time Period':'datetime', 'OBS_VALUE':'population'}, inplace=True)
pop_df['datetime'] = pd.to_datetime(pop_df['datetime'], format='%Y')
pop_df.head()
pop_df = pop_df.set_index('datetime').resample('D', convention='start').interpolate(method='linear')
pop_df.head()

,population
datetime,
2001-01-01,6.530349e+06
2001-01-02,6.530487e+06
2001-01-03,6.530625e+06
2001-01-04,6.530764e+06
2001-01-05,6.530902e+06


In [13]:
# Merge data frames and split into train/test sets. Start with untransformed data then add models with transformations eg sqrt(temp), ln(solar output)
full_df = demand_df.merge(temp_df, on='datetime', how='outer')
full_df = full_df.merge(solar_df, on='datetime', how='outer')
full_df = full_df.merge(pop_df, on='datetime', how='outer')
full_df.sort_index(inplace=True)
full_df.head()

,demand_min,demand_max,temp_min,temp_max,temp_mean,units,cum_units,output,cum_output,population
datetime,,,,,,,,,,
2001-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.530349e+06
2001-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.530487e+06
2001-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.530625e+06
2001-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.530764e+06
2001-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.530902e+06


In [22]:
# Filter dataframe and split into train-test sets
train_mask = (full_df.index >= '2010-01-01') & (full_df.index < '2019-01-01')
test_mask = (full_df.index >= '2019-01-01') & (full_df.index < '2020-12-01') # interpolation didn't fill to the end of 2020 for solar_df

train_df = full_df[train_mask]
test_df = full_df[test_mask]

In [23]:
train_df

,demand_min,demand_max,temp_min,temp_max,temp_mean,units,cum_units,output,cum_output,population
datetime,,,,,,,,,,
2010-01-01,6157.36,8922.42,22.1,28.8,25.094000,1790.000000,21518.000000,2767.019000,3.036293e+04,7.144292e+06
2010-01-02,6112.73,9326.64,21.6,29.4,24.765385,1810.677419,21596.419355,2811.171258,3.049634e+04,7.144495e+06
2010-01-03,6014.91,8277.85,17.9,21.5,19.429825,1831.354839,21674.838710,2855.323516,3.062975e+04,7.144699e+06
2010-01-04,6023.79,9522.30,17.9,23.9,20.625926,1852.032258,21753.258065,2899.475774,3.076316e+04,7.144902e+06
2010-01-05,6287.12,10728.72,15.4,27.7,22.660417,1872.709677,21831.677419,2943.628032,3.089657e+04,7.145106e+06
...,...,...,...,...,...,...,...,...,...,...
2018-12-27,6001.25,11050.31,17.9,32.1,23.083077,4677.258065,485574.903226,34342.086194,1.972439e+06,8.033090e+06
2018-12-28,6269.06,11347.41,19.0,33.6,26.010417,4624.806452,485717.322581,33666.076355,1.973437e+06,8.033333e+06
2018-12-29,6306.77,10910.29,18.6,33.9,25.675000,4572.354839,485859.741935,32990.066516,1.974436e+06,8.033576e+06


In [24]:
test_df

,demand_min,demand_max,temp_min,temp_max,temp_mean,units,cum_units,output,cum_output,population
datetime,,,,,,,,,,
2019-01-01,6162.55,10674.41,21.7,32.8,26.456863,4415.000000,486287.000000,30962.037000,1.977433e+06,8.034305e+06
2019-01-02,6412.74,10978.13,21.3,28.9,23.941667,4460.709677,486475.129032,31341.130548,1.978810e+06,8.034469e+06
2019-01-03,6487.59,10593.87,21.6,27.2,23.968750,4506.419355,486663.258065,31720.224097,1.980188e+06,8.034634e+06
2019-01-04,6450.30,11916.46,19.0,29.8,24.429630,4552.129032,486851.387097,32099.317645,1.981566e+06,8.034798e+06
2019-01-05,6692.19,10882.52,19.3,37.9,23.664865,4597.838710,487039.516129,32478.411194,1.982944e+06,8.034963e+06
...,...,...,...,...,...,...,...,...,...,...
2020-11-26,6051.38,10871.75,14.8,32.5,22.887500,11807.666667,677597.833333,110116.752667,3.490182e+06,8.093864e+06
2020-11-27,6152.19,10173.41,18.6,26.3,21.774510,11792.333333,677988.866667,110417.842533,3.493903e+06,8.093863e+06
2020-11-28,6284.18,12421.83,18.3,40.4,30.079167,11777.000000,678379.900000,110718.932400,3.497624e+06,8.093862e+06


## Linear Regression Models
Now that we have all the data entered, let's build some preliminary linear regression models
- untransformed models
    - add normalisation
- transformed models with normalisation
- add in interactions between variables e.g. temp-solaroutput
... later look at other ML algorithms and data that could improve model

In [25]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [38]:
lm1 = smf.ols('demand_min ~ np.sqrt(temp_min) + np.sqrt(temp_max) + np.log(cum_output) + np.log(population)', data = train_df).fit()

C:\Users\sambo\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [39]:
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             demand_min   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     378.4
Date:                Wed, 22 Mar 2023   Prob (F-statistic):          3.48e-268
Time:                        21:02:44   Log-Likelihood:                -23962.
No. Observations:                3281   AIC:                         4.793e+04
Df Residuals:                    3276   BIC:                         4.796e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -2.719e+04   5956.948     -4.565      0.000   -3.89e+04   -1.55e+04
np.sqrt(temp_min)   -200.3181     10.474    -19.125      0.000    -220.855    -179.781
np.sqrt(temp_max)    -90.4402     16.348     -5.532      0.000    -122.494     -58.387
np.log(cum_output)  -255.6177     15.521    -16.469      0.000    -286.051    -225.185
np.log(population)  2396.4613    388.004      6.176      0.000    1635.706    3157.217
==============================================================================
Omnibus:                      186.785   Durbin-Watson:                   0.389
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              269.029
Skew:                           0.501   Prob(JB):                     3.81e-59
Kurtosis:                       3.983   Cond. No.                     2.05e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""